<a href="https://colab.research.google.com/github/ragch577/RTD-kafka-python--redshift/blob/main/producer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install --upgrade gtfs-realtime-bindings
!pip install gtfs-realtime-bindings==0.0.7
!pip install confluent_kafka
!pip install --user protobuf3-to-dict

In [31]:
import time
from google.protobuf.json_format import MessageToJson
from confluent_kafka import Producer
from google.transit import gtfs_realtime_pb2
from protobuf_to_dict import protobuf_to_dict
#import urllib.request
from urllib.request import Request, urlopen

kafka_topic = 'test'
kafka_producer = Producer({'bootstrap.servers': 'localhost:9092'})
feed = gtfs_realtime_pb2.FeedMessage()
header = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
request = Request('http://www.rtd-denver.com/google_sync/VehiclePosition.pb', headers=header)
response = urlopen(request).read()
#response = urlopen('http://www.rtd-denver.com/google_sync/VehiclePosition.pb',headers=header)
feed.ParseFromString(response)
#feed.response
for entity in feed.entity:
  #print(entity)
  if entity.HasField('vehicle'):
     #print(entity.vehicle)
     update_json = MessageToJson(entity.vehicle)
     print(update_json)
     kafka_producer.produce(
     kafka_topic, update_json.encode('utf-8'))

{
  "position": {
    "latitude": 39.738365,
    "longitude": -104.82385,
    "bearing": 209.0
  },
  "timestamp": "1629710523",
  "vehicle": {
    "id": "1517",
    "label": "1517"
  }
}
{
  "position": {
    "latitude": 39.738766,
    "longitude": -104.823166,
    "bearing": 0.0
  },
  "timestamp": "1629710557",
  "vehicle": {
    "id": "1536",
    "label": "1536"
  }
}
{
  "position": {
    "latitude": 39.769867,
    "longitude": -104.98413,
    "bearing": 203.0
  },
  "timestamp": "1629710180",
  "vehicle": {
    "id": "1544",
    "label": "1544"
  }
}
{
  "position": {
    "latitude": 39.770218,
    "longitude": -104.984634,
    "bearing": 0.0
  },
  "timestamp": "1629710579",
  "vehicle": {
    "id": "1545",
    "label": "1545"
  }
}
{
  "position": {
    "latitude": 39.77025,
    "longitude": -104.9848,
    "bearing": 0.0
  },
  "timestamp": "1629710579",
  "vehicle": {
    "id": "1546",
    "label": "1546"
  }
}
{
  "position": {
    "latitude": 39.770218,
    "longitude": -104

In [3]:
print(len(feed.entity))

45


In [4]:
bus = feed.entity[0]

In [5]:
print(type(bus))

<class 'gtfs_realtime_pb2.FeedEntity'>


In [6]:
print('bus ID:', bus.id, '\n')

bus ID: 1629709901_1533 



In [7]:
bus.vehicle.position

latitude: 39.833465576171875
longitude: -104.68663024902344
bearing: 90.0

In [8]:
bus.vehicle.position.speed

0.0

In [9]:
# convert to dict from our original protobuf feed
buses_dict = protobuf_to_dict(feed)
type(buses_dict)

dict

In [10]:
# get our first bus
bus_again = buses_dict['entity'][0]
bus_again

{'id': '1629709901_1533',
 'vehicle': {'current_status': 2,
  'position': {'bearing': 90.0,
   'latitude': 39.833465576171875,
   'longitude': -104.68663024902344},
  'stop_id': '34651',
  'timestamp': 1629709864,
  'trip': {'direction_id': 0,
   'route_id': 'ATA',
   'schedule_relationship': 0,
   'trip_id': '113758679'},
  'vehicle': {'id': '1533', 'label': '1533'}}}

In [11]:
# We can then use the bus data just as we would a normal dict.

# Get the speed for example
bus_again['vehicle']['position']['bearing']

90.0

In [12]:
import json 


In [13]:
# Serializing json  
json_object = json.dumps(bus_again, indent = 4) 
print(json_object)

{
    "id": "1629709901_1533",
    "vehicle": {
        "trip": {
            "trip_id": "113758679",
            "schedule_relationship": 0,
            "route_id": "ATA",
            "direction_id": 0
        },
        "position": {
            "latitude": 39.833465576171875,
            "longitude": -104.68663024902344,
            "bearing": 90.0
        },
        "current_status": 2,
        "timestamp": 1629709864,
        "stop_id": "34651",
        "vehicle": {
            "id": "1533",
            "label": "1533"
        }
    }
}


In [24]:
def __init__(self):
        with open('.mta_api_key', 'r') as key_in:
            self.api_key = key_in.read().strip()

        #self.mta_api_url = 'http://datamine.mta.info/mta_esi.php?key={}&feed_id=1'.format(
        #     self.api_key)
        self.mta_api_key = 'http://www.rtd-denver.com/google_sync/VehiclePosition.pb'
        self.kafka_topic = 'test'
        self.kafka_producer = Producer({'bootstrap.servers': 'localhost:9092'})

In [25]:
def produce_trip_updates(self):
        feed = gtfs_realtime_pb2.FeedMessage()
        response = requests.get(self.mta_api_url)
        feed.ParseFromString(response.content)

        for entity in feed.entity:
            print(entity)
            if entity.HasField('vehicle'):
                update_json = MessageToJson(entity.vehicle)
                print(update_json)
                self.kafka_producer.produce(
                    self.kafka_topic, update_json.encode('utf-8'))

        self.kafka_producer.flush()

In [26]:
def run(self):
        while True:
            self.produce_trip_updates()
            time.sleep(30)